In [22]:
import pandas as pd
import matplotlib.pyplot as plt

# ---------------------------------
# Part 1: Load and Process the Data
# ---------------------------------

# 1. Load the CSV file (adjust the file path as needed)
df = pd.read_csv('Anantapura Bhairavanthipura_project.csv', 
                 parse_dates=['Date'])

# 2. Rename columns (assuming original columns are 'Monthly' and 'Daily Storage (Selected Period)')
df.columns = ['Date', 'Storage']
print("Original Data:")
print(df.head())

# 3. Extract Year and Month from Date
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
print("\nAfter extracting Year and Month:")
print(df.head())

# 4. Compute Monthly Mean Storage for each (Year, Month)
df['MonthlyMean'] = df.groupby(['Year', 'Month'])['Storage'].transform('mean')
print("\nAfter computing MonthlyMean:")
print(df[['Date', 'Year', 'Month', 'Storage', 'MonthlyMean']].head())

# 5. Compute Yearly Mean Storage for each Year
df['YearlyMean'] = df.groupby('Year')['Storage'].transform('mean')
print("\nAfter computing YearlyMean:")
print(df[['Date', 'Year', 'Storage', 'YearlyMean']].head())

# 6. Compute Discharge (absolute difference between Storage and MonthlyMean)
df['Discharge'] = (df['Storage'] - df['MonthlyMean']).abs()
print("\nAfter computing Discharge (|Storage - MonthlyMean|):")
print(df[['Date', 'Year', 'Month', 'Storage', 'MonthlyMean', 'Discharge']].head())

# 7. Create a new column for the sum of discharge per year (daily discharges summed for the year)
df['YearlySumDischarge'] = df.groupby('Year')['Discharge'].transform('sum')
print("\nAfter adding YearlySumDischarge column:")
print(df[['Date', 'Year', 'Month', 'Discharge', 'YearlySumDischarge']].head())

# 8. Compute yearly D from monthly means: 
#    D = sum of absolute difference between MonthlyMean (for that year & month) and the YearlyMean
# (This was the “regular” D computed earlier)
yearly_D = df.groupby('Year')['MonthlyMean'].mean().reset_index()  # Get one value per year (as an example)
# For D using our earlier method we already computed:
yearly_D_alt = df.groupby('Year').apply(lambda x: (x['MonthlyMean'] - x['YearlyMean']).abs().sum()).reset_index(name='YearlyDischargeFromMonthlyMeans')

# ---------------------------------
# Part 2: Compute Modified Discharge
# ---------------------------------
# Step A: Compute Global Monthly Mean for each month (over all years)
global_monthly = df.groupby('Month')['Storage'].mean().reset_index()
global_monthly.rename(columns={'Storage': 'GlobalMonthlyMean'}, inplace=True)
print("\nGlobal Monthly Means (over all years):")
print(global_monthly)

# Step B: Get unique monthly means for each Year & Month 
# (Since df has daily records, the MonthlyMean is repeated. We extract one row per Year-Month.)
yearly_monthly = df.groupby(['Year', 'Month']).first().reset_index()
# Retain only Year, Month, and MonthlyMean
yearly_monthly = yearly_monthly[['Year', 'Month', 'MonthlyMean']]

# Step C: Merge Global Monthly Mean with yearly monthly data
yearly_monthly = pd.merge(yearly_monthly, global_monthly, on='Month', how='left')

# Step D: Compute the difference for each Year-Month:
yearly_monthly['Difference'] = yearly_monthly['MonthlyMean'] - yearly_monthly['GlobalMonthlyMean']
# (If you need absolute differences, you can use .abs() here)
# For now, we use the raw difference as in S2015 - Sjan(bar)

# Step E: For each year, sum the difference over the 12 months to get the modified discharge D_modified
modified_discharge = yearly_monthly.groupby('Year')['Difference'].sum().reset_index()
modified_discharge.rename(columns={'Difference': 'ModifiedDischarge'}, inplace=True)
# print(modefied_discharge)

print("\nModified Discharge (sum over months of [MonthlyMean - GlobalMonthlyMean]) for each year:")
print(modified_discharge)

# Find maximum and minimum modified discharge and their corresponding years
max_modified = modified_discharge['ModifiedDischarge'].max()
min_modified = modified_discharge['ModifiedDischarge'].min()
max_modified_year = modified_discharge.loc[modified_discharge['ModifiedDischarge'].idxmax(), 'Year']
min_modified_year = modified_discharge.loc[modified_discharge['ModifiedDischarge'].idxmin(), 'Year']

print(f"\nMaximum Modified Discharge = {max_modified} in Year {max_modified_year}")
print(f"Minimum Modified Discharge = {min_modified} in Year {min_modified_year}")




Original Data:
        Date  Storage
0 2023-11-01      NaN
1 2023-11-02      NaN
2 2023-11-03      NaN
3 2023-11-04      NaN
4 2023-11-05      NaN

After extracting Year and Month:
        Date  Storage  Year  Month
0 2023-11-01      NaN  2023     11
1 2023-11-02      NaN  2023     11
2 2023-11-03      NaN  2023     11
3 2023-11-04      NaN  2023     11
4 2023-11-05      NaN  2023     11

After computing MonthlyMean:
        Date  Year  Month  Storage  MonthlyMean
0 2023-11-01  2023     11      NaN          NaN
1 2023-11-02  2023     11      NaN          NaN
2 2023-11-03  2023     11      NaN          NaN
3 2023-11-04  2023     11      NaN          NaN
4 2023-11-05  2023     11      NaN          NaN

After computing YearlyMean:
        Date  Year  Storage  YearlyMean
0 2023-11-01  2023      NaN         NaN
1 2023-11-02  2023      NaN         NaN
2 2023-11-03  2023      NaN         NaN
3 2023-11-04  2023      NaN         NaN
4 2023-11-05  2023      NaN         NaN

After computing Disch

In [19]:
import os
import glob
import pandas as pd
import shutil

# Base folder paths
base_dir = '/Users/deepakchaurasia/Desktop/UGP_1'      # Folder with state subfolders containing original CSVs
combined_base_dir = '/Users/deepakchaurasia/Desktop/Combined_UGP_1'  # Folder to save combined files

# Final summary CSV path (will contain only Reservoir and Modified Max Value)
summary_csv_path = os.path.join(combined_base_dir, 'reservoir_max_modified.csv')

# Remove old combined folder to start fresh
if os.path.exists(combined_base_dir):
    shutil.rmtree(combined_base_dir)
os.makedirs(combined_base_dir, exist_ok=True)

# Global Counters (for debugging)
total_original_files = 0
total_combined_files = 0
total_processed_files = 0

# -------------------------------------------
# Helper function to extract and normalize reservoir info
# -------------------------------------------
def parse_reservoir_info(filename):
    """
    Extracts and normalizes district and reservoir names from a filename.
    Accepts formats like:
      "District_Reservoir from YYYY-MM to YYYY-MM.csv" or
      "District Reservoir from YYYY-MM to YYYY-MM.csv"
    Returns a key string in the form "District Reservoir".
    """
    name_no_ext = filename.replace('.csv', '').strip()
    main_part = name_no_ext.split(' from ')[0].strip()  # Remove date range part
    # Try splitting by underscore first; if not, split by whitespace.
    if '_' in main_part:
        parts = main_part.split('_', 1)
    else:
        parts = main_part.split(' ', 1)
    
    if len(parts) == 2:
        district, reservoir = parts[0].strip(), parts[1].strip()
    else:
        district = parts[0].strip()
        reservoir = "Unknown"
    return f"{district} {reservoir}"

# -------------------------------------------
# Step 1: Combine Files with the Same Reservoir Name
# -------------------------------------------
for state in os.listdir(base_dir):
    state_path = os.path.join(base_dir, state)
    if not os.path.isdir(state_path):
        continue  # Skip non-directory items

    # Create an output folder for the state within the combined folder
    combined_state_path = os.path.join(combined_base_dir, state)
    os.makedirs(combined_state_path, exist_ok=True)

    # Get list of CSV files in the state folder
    csv_files = glob.glob(os.path.join(state_path, '*.csv'))
    total_original_files += len(csv_files)

    # Group files by reservoir key
    reservoir_groups = {}
    for file in csv_files:
        filename = os.path.basename(file)
        key = parse_reservoir_info(filename)
        reservoir_groups.setdefault(key, []).append(file)

    # Process each reservoir group – combine only if exactly 2 files are present
    for key, files in reservoir_groups.items():
        if len(files) != 2:
            print(f"Skipping reservoir '{key}' in state '{state}': found {len(files)} file(s) (expected 2).")
            continue

        combined_file_path = os.path.join(combined_state_path, f"{key}.csv")
        df_list = []
        for f in files:
            try:
                temp_df = pd.read_csv(f, parse_dates=['Monthly'])
                # Rename columns for consistency
                temp_df = temp_df.rename(columns={
                    'Monthly': 'Date',
                    'Daily Storage (Selected Period)': 'Storage'
                })
                df_list.append(temp_df)
            except Exception as e:
                print(f"Error reading file {f}: {e}")

        if not df_list:
            continue

        combined_df = pd.concat(df_list, ignore_index=True)
        combined_df['Date'] = pd.to_datetime(combined_df['Date'], errors='coerce')
        combined_df = combined_df.dropna(subset=['Date'])
        combined_df.to_csv(combined_file_path, index=False)
        total_combined_files += 1
        print(f"Combined file created for reservoir '{key}' in state '{state}'.")

# -------------------------------------------
# Step 2: Calculate Modified Maximum Discharge for Each Combined Reservoir
# -------------------------------------------
# This step follows the logic:
# 1. Compute MonthlyMean storage per (Year, Month)
# 2. Compute GlobalMonthlyMean (over all years) for each month.
# 3. For each (Year, Month), compute Difference = MonthlyMean - GlobalMonthlyMean.
# 4. Sum these differences for each year = ModifiedDischarge.
# 5. Take the maximum ModifiedDischarge value (and its corresponding year) for the reservoir.
summary_results = []
for state in os.listdir(combined_base_dir):
    state_path = os.path.join(combined_base_dir, state)
    if not os.path.isdir(state_path):
        continue

    csv_files = glob.glob(os.path.join(state_path, '*.csv'))
    total_processed_files += len(csv_files)

    for file in csv_files:
        try:
            df = pd.read_csv(file, parse_dates=['Date'])
            if 'Storage' not in df.columns:
                print(f"File {file} missing 'Storage' column. Skipping.")
                continue
            df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
            df = df.dropna(subset=['Date'])
            df['Year'] = df['Date'].dt.year
            df['Month'] = df['Date'].dt.month

            # Compute Monthly Mean Storage per (Year, Month)
            df['MonthlyMean'] = df.groupby(['Year', 'Month'])['Storage'].transform('mean')
            
            # Compute Global Monthly Mean for each month over all years
            global_monthly = df.groupby('Month')['Storage'].mean().reset_index().rename(
                columns={'Storage': 'GlobalMonthlyMean'}
            )
            
            # For each (Year, Month), get one record (first row)
            yearly_monthly = df.groupby(['Year', 'Month']).first().reset_index()
            yearly_monthly = yearly_monthly[['Year', 'Month', 'MonthlyMean']]
            
            # Merge with Global Monthly Mean
            yearly_monthly = pd.merge(yearly_monthly, global_monthly, on='Month', how='left')
            
            # Compute Difference for each Year-Month
            yearly_monthly['Difference'] = yearly_monthly['MonthlyMean'] - yearly_monthly['GlobalMonthlyMean']
            
            # Sum differences over each year to get Modified Discharge per year
            modified_discharge = yearly_monthly.groupby('Year')['Difference'].sum().reset_index().rename(
                columns={'Difference': 'ModifiedDischarge'}
            )
            
            # Get the maximum Modified Discharge value and corresponding year
            if not modified_discharge.empty:
                max_modified = modified_discharge['ModifiedDischarge'].max()
                max_modified_year = modified_discharge.loc[modified_discharge['ModifiedDischarge'].idxmax(), 'Year']
            else:
                max_modified = 0
                max_modified_year = 0

            # Store only the reservoir key and its modified max value
            reservoir_name = os.path.basename(file).replace('.csv', '')
            summary_results.append({
                'State': state,
                'Reservoir': reservoir_name,
                'Modified Max Value': max_modified,
                'Year': max_modified_year
            })
            print(f"Processed {state} - {reservoir_name}: Modified Max Value = {max_modified} (Year: {max_modified_year})")
        except Exception as e:
            print(f"Error processing {file}: {e}")

# -------------------------------------------
# Step 3: Save the Summary CSV (Only storing the modified max value)
# -------------------------------------------
summary_df = pd.DataFrame(summary_results)
# Sort by Reservoir name if desired
summary_df = summary_df.sort_values('Reservoir').reset_index(drop=True)
summary_df.to_csv(summary_csv_path, index=False)

# -------------------------------------------
# Final Debugging & Count Check
# -------------------------------------------
print("\nProcessing complete.")
print(f"Total original CSV files in '{base_dir}': {total_original_files}")
print(f"Total combined reservoir files created in '{combined_base_dir}': {total_combined_files} (Expected: {total_original_files // 2})")
print(f"Total combined files processed for calculation: {total_processed_files}")
print(f"Final summary CSV saved at: {summary_csv_path}")


Combined file created for reservoir 'Anantapura Merala_reservior' in state 'AndhraPradesh'.
Combined file created for reservoir 'Guntur Buggavagu' in state 'AndhraPradesh'.
Combined file created for reservoir 'EastGodavari Yeluru_RESERVIOR' in state 'AndhraPradesh'.
Combined file created for reservoir 'YSR Thimapuram_SR1' in state 'AndhraPradesh'.
Combined file created for reservoir 'EastGodavari Subbaredy_Sagar' in state 'AndhraPradesh'.
Combined file created for reservoir 'Vishakhapatnam Raiwada Reservior' in state 'AndhraPradesh'.
Combined file created for reservoir 'Anantapura Jeedipalli_reservior' in state 'AndhraPradesh'.
Combined file created for reservoir 'YSR Veligallu' in state 'AndhraPradesh'.
Combined file created for reservoir 'westGodavari Kovvadakalva Reservior' in state 'AndhraPradesh'.
Combined file created for reservoir 'Prakasam Gundlakamma_reservior' in state 'AndhraPradesh'.
Combined file created for reservoir 'EastGodavari Yeluru_reservior' in state 'AndhraPradesh